In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# المكتبات
import os
import cv2
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
import re
import shutil

In [4]:
# Define the source dataset paths
dataset_path = "/content/drive/MyDrive/nbtar/data/soil_types_dataset"
train_source = os.path.join(dataset_path, "Train")
test_source = os.path.join(dataset_path, "test")

# Define the destination directory
new_dataset_path = "/content/dataset"

# Create new train and test directories in the new location
train_dest = os.path.join(new_dataset_path, "Train")
test_dest = os.path.join(new_dataset_path, "Test")

os.makedirs(train_dest, exist_ok=True)
os.makedirs(test_dest, exist_ok=True)
# Function to rename and move the images
def process_image(source_folder, destination_folder,image_counter=1):

    for soil in os.listdir(source_folder):
        soil_folder = os.path.join(source_folder, soil)
        new_soil_folder = os.path.join(destination_folder, soil)
        os.makedirs(new_soil_folder, exist_ok=True)

        if os.path.isdir(soil_folder):
            # Initialize a counter for numbering images

            for filename in os.listdir(soil_folder):
                old_path = os.path.join(soil_folder, filename)

                # Skip directories like .ipynb_checkpoints
                if os.path.isdir(old_path):
                    continue

                # Get the file extension
                _, ext = os.path.splitext(filename)
                if ext.lower() not in ['.jpg', '.jpeg', '.png']:  # Ensure the file is an image
                    continue

                # Rename the file with just numbers (1, 2, 3, ...)
                new_filename = f"{image_counter}{ext}"
                new_path = os.path.join(new_soil_folder, new_filename)

                # Rename & move the file
                shutil.copy2(old_path, new_path)
                print(f"Copied: {filename} -> {new_path}")

                # Increment the counter for the next image
                image_counter += 1
    return image_counter

# Process training images
image_counter=process_image(train_source, train_dest)

# Process testing images
process_image(test_source, test_dest,image_counter)

print(f" New dataset saved in: {new_dataset_path}")


Copied: 1000_F_233167094_JhQz9xvHtlN5UyT3zt748JiJolOSid7s.jpg -> /content/dataset/Train/Red soil/1.jpg
Copied: 15kg-red-soil-500x500.png -> /content/dataset/Train/Red soil/2.png
Copied: Copy of 61050549-red-soil-texture-background.png -> /content/dataset/Train/Red soil/3.png
Copied: Copy of Sample6.30.jpg -> /content/dataset/Train/Red soil/4.jpg
Copied: Copy of Sample6.60.jpg -> /content/dataset/Train/Red soil/5.jpg
Copied: Copy of Sample8.120.jpg -> /content/dataset/Train/Red soil/6.jpg
Copied: Copy of Sample6.180.jpg -> /content/dataset/Train/Red soil/7.jpg
Copied: Copy of 61AysVsqgL.jpg -> /content/dataset/Train/Red soil/8.jpg
Copied: Copy of Sample8.60.jpg -> /content/dataset/Train/Red soil/9.jpg
Copied: Copy of 15kg-red-soil-500x500.png -> /content/dataset/Train/Red soil/10.png
Copied: Copy of Sample6.90.jpg -> /content/dataset/Train/Red soil/11.jpg
Copied: Copy of Sample6.150.jpg -> /content/dataset/Train/Red soil/12.jpg
Copied: Copy of 51em0GmiApL._SX466_.jpg -> /content/dataset

In [5]:
# Load and sort training folder classes
soils_train = sorted(os.listdir("/content/dataset/Train"))
print("Train Classes:", soils_train)

# Load and sort validation folder classes
soils_validations = sorted(os.listdir("/content/dataset/Test"))
print("Validation Classes:", soils_validations)


Train Classes: ['Alluvial soil', 'Black Soil', 'Clay soil', 'Red soil']
Validation Classes: ['Alluvial soil', 'Black Soil', 'Clay soil', 'Red soil']


In [6]:
soils_train_images = {}
for soil in soils_train:
    soils_train_images[soil] = os.listdir(f"/content/dataset/Train/{soil}")

In [7]:
for soil in soils_train_images:
    print(f"Dataset contains {len(soils_train_images[soil])} images of {soil}")

Dataset contains 523 images of Alluvial soil
Dataset contains 228 images of Black Soil
Dataset contains 197 images of Clay soil
Dataset contains 267 images of Red soil


In [50]:
# Define paths البيانات لصور التربة
train_data_dir = '/content/dataset/Train'
test_data_dir = '/content/dataset/Test'

# Function to load images
def load_images_from_folder(folder):
    images = []  # مصفوفة للصور
    labels = []  # مصفوفة للأسماء

    for label in os.listdir(folder):
        label_path = os.path.join(folder, label)
        if not os.path.isdir(label_path):  # تأكد أنه مجلد وليس ملف
            continue

        for image_name in os.listdir(label_path):
            image_path = os.path.join(label_path, image_name)
            image = cv2.imread(image_path)
            if image is None:  # تحقق من أن الصورة لم تفشل في التحميل
                print(f"Warning: Couldn't read image {image_path}")
                continue

            image = cv2.resize(image, (64, 64))  # تصغير الصور إلى 64x64
            images.append(image)
            labels.append(label)

    return np.array(images), np.array(labels)

# تحميل البيانات
X_train, y_train = load_images_from_folder(train_data_dir)
X_test, y_test = load_images_from_folder(test_data_dir)

# تحويل التصنيفات إلى أرقام
label_to_index = {label: idx for idx, label in enumerate(np.unique(y_train))}
index_to_label = {idx: label for label, idx in label_to_index.items()}

y_train = np.array([label_to_index[label] for label in y_train])
y_test = np.array([label_to_index[label] for label in y_test])

# تحويل التصنيفات إلى One-Hot Encoding
#have 4 classes {'Alluvial soil': 0, 'Black Soil': 1, 'Clay soil': 2, 'Red soil': 3}
y_train = to_categorical(y_train, num_classes=len(label_to_index))
y_test = to_categorical(y_test, num_classes=len(label_to_index))

# تطبيع الصور
X_train = X_train.astype('float32') / 255.0
X_test = X_test.astype('float32') / 255.0

# بناء نموذج CNN
model_cnn = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 3)),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(len(label_to_index), activation='softmax')  # سوفت ماكس لتصنيف متعدد
])

# تجميع النموذج
model_cnn.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# تدريب النموذج
model_cnn.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))  #


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
38/38 ━━━━━━━━━━━━━━━━━━━━ 5s 58ms/step - accuracy: 0.5185 - loss: 1.1690 - val_accuracy: 0.7500 - val_loss: 0.6795
Epoch 2/10
38/38 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - accuracy: 0.7820 - loss: 0.5900 - val_accuracy: 0.8441 - val_loss: 0.4523
Epoch 3/10
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.8624 - loss: 0.3883 - val_accuracy: 0.8765 - val_loss: 0.3104
Epoch 4/10
38/38 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.8691 - loss: 0.3317 - val_accuracy: 0.8882 - val_loss: 0.3041
Epoch 5/10
38/38 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.8911 - loss: 0.2681 - val_accuracy: 0.9529 - val_loss: 0.1867
Epoch 6/10
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.9204 - loss: 0.2475 - val_accuracy: 0.9529 - val_loss: 0.1759
Epoch 7/10
38/38 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.9313 - loss: 0.1997 - val_accuracy: 0.9206 - val_loss: 0.2233
Epoch 8/10
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.9336 - loss: 0.2058 - val_accuracy: 0.9765 - val_los

In [54]:
# Save the model as h5
model_cnn.save("soil_classifier.h5")

# # Convert to TFLite
converter = tf.lite.TFLiteConverter.from_keras_model(model_cnn)
tflite_model = converter.convert()

# Save the converted model
with open("soil_classifierls.tflite", "wb") as f:
    f.write(tflite_model)

print("Model converted successfully!")


Saved artifact at '/tmp/tmp9skwx3sm'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 64, 64, 3), dtype=tf.float32, name='keras_tensor_170')
Output Type:
  TensorSpec(shape=(None, 4), dtype=tf.float32, name=None)
Captures:
  137421708367184: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137421708363152: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137421708364304: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137421708364496: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137421708363344: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137421708365264: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137421708365648: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137421708376976: TensorSpec(shape=(), dtype=tf.resource, name=None)
Model converted successfully!


In [53]:
print(label_to_index)

{'Alluvial soil': 0, 'Black Soil': 1, 'Clay soil': 2, 'Red soil': 3}
